In [19]:
import pandas as pd
from pathlib import Path

pd.set_option("display.max_rows", 4)

d_root = Path("/home/fpavlov/projects/article_conserved_miRNA")
d_data = d_root / "data"
d_img = d_root / "img"


# Input
f_flipon_to_data = d_data / "flipon_to_data.tsv"
f_flipon_to_data_og = d_data / "flipon_to_data_og.tsv"

# output
!mkdir {d_data}/ideogram_data

In [12]:
def get_category(data_input: pd.Series):
    nans = data_input.isna()
    if nans[1] and not nans[2]:
        return f"{data_input['group']}_repeats.bed"
    elif not nans[1] and nans[2]:
        return f"{data_input['group']}_ccre.bed"
    return None

data = pd.read_table(f_flipon_to_data).assign(
    flipon_category=lambda x: x[
        ["group", "cCRE (+-200bp slop)", "LINE/LTR repeats (+-200bp slop)"]
    ].apply(get_category, axis=1)
)

data


,group,coordinates,miRNA (+ strand),miRNA (- strand),miRNA (intersection),annotation,gene_info,cCRE (+-200bp slop),LINE/LTR repeats (+-200bp slop),flipon_category
0,sidd,chr10:100119606-100119743,"miR-539, miR-670","miR-143, miR-186, miR-30",NaN,Distal Intergenic,Gm22918 (+),NaN,"LTR@RLTR10C (+), LINE@Lx8b (-), LTR@MTB (+)",sidd_repeats.bed
1,sidd,chr10:100146768-100146986,"miR-155, miR-374",miR-448,NaN,Promoter (2-3kb),Gm25287 (+),NaN,LTR@MTB_Mm (-),sidd_repeats.bed
...,...,...,...,...,...,...,...,...,...,...
31684,h-dna,chrY:3865190-3865212,miR-122,NaN,NaN,Distal Intergenic,Gm8521 (+),NaN,NaN,None
31685,h-dna,chrY:3879642-3879666,miR-185,NaN,NaN,Distal Intergenic,Gm18177 (-),NaN,NaN,None


In [20]:
for idx, group_df in data.groupby('flipon_category'):
    print(idx)
    out_df = group_df['coordinates'].str.split(r':|-', regex=True, expand=True)
    out_df.to_csv(d_data / "ideogram_data" / f"ideogram_data_mirna_{idx}", sep='\t', index=False, header=False)

g4_ccre.bed
g4_repeats.bed
h-dna_ccre.bed
h-dna_repeats.bed
sidd_ccre.bed
sidd_repeats.bed
z-dna_ccre.bed
z-dna_repeats.bed
